In [1]:
from __future__ import print_function
import torch as t
from torch import nn

In [7]:
a = t.randn(5)
a

tensor([ 0.0253,  0.4461, -0.5702, -2.3491,  0.1095])

In [ ]:
from PIL import Image
from torchvision.transforms import ToTensor,ToPILImage

to_tensor = ToTensor()
to_pil = ToPILImage()

img = Image.open('./img/lena.png')
img

In [ ]:
input = to_tensor(img)
print(input.size())
input.unsqueeze_(0) # 增加维度
print(input.size())

# 锐化卷积核
kernel = t.ones(3,3)/-9
kernel[1][1] = 1
conv = nn.Conv2d(1,1,(3,3),1,bias=False)
conv.weight.data = kernel.view(1,1,3,3)

out = conv(input)
to_pil(out.data.squeeze(0))

In [ ]:
pool = nn.AvgPool2d(2,2)
out = pool(input)
to_pil(out.data.squeeze(0))

**循环神经网络层(RNN)**

In [19]:
t.manual_seed(1000)
input = t.randn(2, 3, 4) # 输入：batch_size=3，序列长度都为2，序列中每个元素占4维
lstm = nn.LSTM(4,3,1) # lstm输入向量4维，隐藏元3，1层
# 初始状态：1层，batch_size=3，3个隐藏元
h0 = t.randn(1, 3, 3)
h0 = t.randn(1, 3, 3)
output,hn = lstm(input,(h0,c0))

tensor([[[-0.5306, -1.1300, -0.6734, -0.7669],
         [-0.7029,  0.9896, -0.4482,  0.8927],
         [-0.6043,  1.0726,  1.0481,  1.0527]],

        [[-0.6424, -1.2234, -1.0794, -0.6037],
         [-0.7926, -0.1414, -1.0225, -0.0482],
         [ 0.6610, -0.8908,  1.4793, -0.3934]]])

**优化器**

In [8]:
class Net(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        
        self.features = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )
        
    def forward(self,x):
        x = self.features(x)
        x = x.view(-1,16*5*5)
        x = self.classifier(x)
        return x
    
net = Net()
net

Net(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [7]:
from torch import optim

optimizer = optim.SGD(params=net.parameters(),lr=1)
net.zero_grad() # optimier.zero_grad()

input = t.randn(1,3,32,32)
output = net(input)
output.backward(output)

optimizer.step()

In [ ]:
# 为不同子网络设置不同的学习率，在finetune中经常用到
# 如果对某个参数不指定学习率，就使用最外层的默认学习率
optimizer = optim.SGD([
    {'params':net.features.parameters()},
    {'params':net.classifier.parameters(),'lr':1e-2}
],lr=1e-5)
optimizer

In [ ]:
# 只为两个全连接层设置较大的学习率，其余层的学习率较小
specaial_layers = nn.ModuleList([
    net.classifier[0],
    net.classifier[2]
]) # layer List的是ModuleList

specaial_layers_params = list(map(id,specaial_layers.parameters()))
# print(len(list(specaial_layers.parameters())))
# specaial_layers_params
base_params = filter(lambda p:id(p) not in specaial_layers_params,
                    net.parameters())
optimizer = optim.SGD([
    {'params':base_params},
    {'params':specaial_layers.parameters(),'lr':0.01}
],lr=0.001)

**调整学习率**
- 修改optimizer.param_groups中对应的学习率
- 新建优化器（对于使用动量的优化器（如Adam），会丢失动量等状态信息，可能会造成损失函数的收敛出现震荡等情况）

In [ ]:
# 方法1: 调整学习率，新建一个optimizer
old_lr = 0.1
optimizer1 =optim.SGD([
                {'params': net.features.parameters()},
                {'params': net.classifier.parameters(), 'lr': old_lr*0.1}
            ], lr=1e-5)
optimizer1

In [ ]:
# 方法2: 调整学习率, 手动decay, 保存动量
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1
    
optimizer

**nn.functional**
- 模型有可学习的参数，最好用nn.Module
- 虽然dropout操作也没有可学习操作，但建议还是使用nn.Dropout而不是nn.functional.dropout
- 对于有可学习参数的模块，也可以用functional来代替，需要手动定义参数parameter

**保存模型**

In [ ]:
# 保存模型
t.save(net.state_dict(), 'net.pth')

# 加载已保存的模型
net2 = Net()
net2.load_state_dict(t.load('net.pth'))